In [1]:
%cd /ibex/user/slimhy/PADS/code
%reload_ext autoreload
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import torch

import util.misc as misc
import models.s2vs as ae_mods


def get_args_parser():
    parser = argparse.ArgumentParser("Extracting Features", add_help=False)

    # Model parameters
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU"
        " (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--intensity_loss",
        action="store_true",
        default=False,
        help="Contrastive edit intensity loss using ground-truth labels.",
    )

    # Dataset parameters
    parser.add_argument(
        "--dataset",
        type=str,
        choices=["graphedits"],
        help="dataset name",
    )
    parser.add_argument(
        "--data_path",
        type=str,
        help="dataset path",
    )
    parser.add_argument(
        "--data_type",
        type=str,
        help="dataset type",
    )
    parser.add_argument(
        "--max_edge_level",
        default=None,
        type=int,
        help="maximum edge level to use",
    )
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient "
        "(sometimes) transfer to GPU.",
    )

    return parser


# Set dummy arg string to debug the parser
call_string = """--ae_pth ckpt/ae_m512.pth \
    --ae kl_d512_m512_l8 \
    --ae-latent-dim 4096 \
    --data_path /ibex/project/c2273/PADS/3DCoMPaT \
    --batch_size 32 \
    --num_workers 8 \
    --device cuda"""
    

# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())

# --------------------
device = torch.device(args.device)

# Fix the seed for reproducibility
misc.set_all_seeds(args.seed)

torch.backends.cudnn.benchmark = True
# --------------------

# Instantiate autoencoder
ae = ae_mods.__dict__[args.ae]()
ae.eval()
print("Loading autoencoder %s" % args.ae_pth)
ae.load_state_dict(torch.load(args.ae_pth, map_location="cpu")["model"])
ae = ae.to(device)

# Compile using torch.compile
ae = torch.compile(ae, mode="max-autotune")

/ibex/user/slimhy/PADS/code
Set seed to 0
Loading autoencoder ckpt/ae_m512.pth


In [2]:
from datasets.latents import ShapeLatentDataset, ComposedPairedShapesLoader

class PairType():
    NO_ROT_PAIR = "rand_no_rot,rand_no_rot"
    PART_DROP = "part_drop,orig"

# Create your datasets
dataset_train = ShapeLatentDataset(args.data_path, split="train", shuffle_parts=True)
dataset_val = ShapeLatentDataset(args.data_path, split="test", shuffle_parts=False)

# Create the DataLoader using the sampler
data_loader_train = ComposedPairedShapesLoader(
    dataset_train,
    batch_size=args.batch_size,
    pair_types_list=[PairType.PART_DROP, PairType.NO_ROT_PAIR],
    num_workers=0,
    shuffle=True,
    use_distributed=False
) 

In [3]:
from models.partvae import PartAwareVAE

pvae = PartAwareVAE(
    dim=512,
    latent_dim=128,
    heads=8,
    dim_head=64,
    depth=2,
).to(device)
pvae = pvae.to(device)

In [4]:
# Print param count in human readable format
print("Model param count: ", misc.count_params(pvae))

# Print param count in human readable format
print("Model param count: ", misc.count_params(ae))

Model param count:  20,171,080
Model param count:  106,128,913


In [5]:
from losses.partvae import KLRecLoss, ScaleInvariantLoss, PartDropLoss
from torch.nn import functional as F


# Instantiate the losses
mse_loss = torch.nn.MSELoss()

kl_rec_loss = KLRecLoss()
scale_inv_loss = ScaleInvariantLoss()
part_drop_loss = PartDropLoss()

# Use the dataloader in your training loop
for pair_types, (l_a, bb_a, bb_l_a, meta_a), (l_b, bb_b, bb_l_b, meta_b) in data_loader_train:
    device = pvae.device

    # Compute the mask from batch labels
    mask_a = (bb_l_a != -1).to(device)  # B x 24
    mask_b = (bb_l_b != -1).to(device)  # B x 24

    l_a, l_b = l_a.to(device), l_b.to(device)  # B x 512 x 8
    bb_a, bb_b = bb_a.to(device), bb_b.to(device)  # B x 24 x 4 x 3
    bb_l_a, bb_l_b = bb_l_a.to(device), bb_l_b.to(device)  # B x 24

    # Forward passes
    logits_a, kl_a, part_latents_a = pvae(
        latents=l_a, part_bbs=bb_a, part_labels=bb_l_a, batch_mask=mask_a
    )
    logits_b, kl_b, part_latents_b = pvae(
        latents=l_b, part_bbs=bb_b, part_labels=bb_l_b, batch_mask=mask_b
    )

    # KL Reg loss
    kl_reg = kl_rec_loss(kl_a, mask=mask_a) + kl_rec_loss(kl_b, mask=mask_b)

    # L2 loss
    rec_loss = F.mse_loss(logits_a, l_a) + F.mse_loss(logits_b, l_b)

    if pair_types == PairType.NO_ROT_PAIR:
        inv_loss = scale_inv_loss(part_latents_a, part_latents_b, mask_a)
    elif pair_types == PairType.PART_DROP:
        inv_loss = part_drop_loss(
            part_latents_a, part_latents_b, bb_a, bb_b, mask_a, mask_b
        )
    break